In [ ]:
!pip install -r requirements.txt --quiet

## Download Raw Data
### 1. EDGAR Filings (Securities Agreements)
Contains actual agreement language and compliance clauses from SEC filings. These documents demonstrate how regulations are implemented in practice through material contracts, exhibits, and disclosure documents.

### 2. SEC Regulations (Authoritative Rules)
Provides the official regulatory text and requirements from the Code of Federal Regulations. These define the legal standards and obligations that agreements must comply with, covering areas such as private placements, restricted securities resales, and anti-fraud provisions.

### 3. Case Law (Judicial Interpretations)
Judicial interpretations - how regulations are applied in real-world scenarios. These cases provide context for understanding regulatory requirements and compliance validation. 

In [9]:
# 1. EDGAR Filings (Securities Agreements)
from utils.download_raw_data import download_edgar_agreements

download_edgar_agreements(output_path="data/raw/cases")

  8-K: Found 99 filings to check
    Downloaded: 20
  10-K: Found 8 filings to check
  10-Q: Found 11 filings to check
  S-1: Found 10 filings to check
    Downloaded: 40
  S-3: Found 5 filings to check
  DEF 14A: Found 4 filings to check
  20-F: Found 2 filings to check
  F-1: Found 5 filings to check
  424B: Found 200 filings to check
  D: Found 90 filings to check
✓ EDGAR: 44 agreements from 10 form types, 0 errors


In [10]:
# 2. SEC Regulations (Authoritative Rules)
from utils.download_raw_data import download_sec_regulations

download_sec_regulations(output_path="data/raw/regulations")

  Regulations: 10
  Regulations: 20
  Regulations: 30
✓ Regulations: 38 rules, 0 errors


In [ ]:
# 3. Case Law (Judicial Interpretations)
from utils.download_raw_data import download_courtlistener_cases
import os

os.environ["courtlistener_api_key"] = "" #Get your's at https://www.courtlistener.com/register/?next=/!3F

download_courtlistener_cases(api_key=os.environ["courtlistener_api_key"], max_cases=5000, output_path="data/raw/cases")

## Synthetic Data Generation - GroundTruth for Fine-tuning Nova
The training data is built from three interconnected information sources that together provide comprehensive coverage of securities law: EDGAR Filings (Securities Agreements), SEC Regulations (Authoritative Rules), Case Law (Judicial Interpretations)

Data is generate using [Synthetic Data Generation Kit](https://github.com/meta-llama/synthetic-data-kit) from Meta to create training examples.

- **Model Routing**: A local Flask server (`aws_proxy.py`) routes API requests from the kit to AWS services (SageMaker endpoints and Bedrock models), bypassing the kit's API key requirement and authenticating via [AWS SigV4](https://docs.aws.amazon.com/AmazonS3/latest/API/sig-v4-authenticating-requests.html) using a signature created by using access keys (access key ID, secret access key.
- **Prompt Engineering**: Custom prompts can be provided in the kit's or user-provided configuration file (`aws-config.yaml`). Follow model-specific best practices for prompting to ensure high-quality generation
- **Flexible Configuration**: `aws_proxy.py` supports multiple AWS options - Bedrock models via model ID or inference profile ID, and SageMaker real-time endpoints via deployed model name


![alt text](imgs/data_prep.gif)

### Nova Fine-Tuning Format for SFT

[Nova Prompting Best Practices](https://docs.aws.amazon.com/nova/latest/userguide/prompting-text-understanding.html) ; 
[Nova SFT Fine-Tuning Data Format](https://docs.aws.amazon.com/nova/latest/userguide/fine-tune-prepare-data-understanding.html)

The first turn in messages should always start with "role": "user". The last turn is the bot's response, denoted by "role": "assistant".

Each example should be on a single line in your JSONL file, with one JSON object per line. Sample data format.


```json
{
    "system": [{"text": "Content of the System prompt"}],
    "messages": [
        {
            "role": "user",
            "content": ["text": "Content of the user prompt"]
        },
        {
            "role": "assistant",
            "content": ["text": "Content of the assistant answer"]
        }
    ]
}
```

### Nova Fine-Tuning Format for RFT

[Nova Prompting Best Practices](https://docs.aws.amazon.com/nova/latest/userguide/prompting-text-understanding.html) ; 
[Nova RFT Fine-Tuning Data Format](https://docs.aws.amazon.com/nova/latest/userguide/fine-tune-prepare-data-understanding.html)

Each example has a unique 'id', the user, assistant and system prompts are keyed a bit differently.

Each example should be on a single line in your JSONL file, with one JSON object per line. Sample data format.


```json
{
    "id": "00001",
    "messages": [
        {
            "role": "system",
            "content": "Content of the System prompt"
        },            
        {
            "role": "user",
            "content": "Content of the user prompt"
        }
    ],
    "reference_answer": {
            "Content of the assistant answer"
    }
}
```

### 🎯 Query Types

Models learn to classify queries into 8 predefined categories:

| Type | Tools Used | Description |
|------|------------|-------------|
| `regulatory_definition` | statute_retrieval | Single regulatory lookup |
| `judicial_interpretation` | case_law_search | Court precedent research |
| `compliance_validation` | compliance_checker | Agreement clause validation |
| `citation_verification` | citation_validator | Legal citation checking |
| `regulatory_compliance_analysis` | statute_retrieval → compliance_checker | Multi-step regulatory analysis |
| `judicial_compliance_assessment` | case_law_search → compliance_checker | Court precedent + validation |
| `cross_document_analysis` | statute_retrieval → case_law_search → compliance_checker | Full 3-source analysis |
| `regulatory_interpretation_research` | statute_retrieval → case_law_search | Regulation + precedent research |



### 🛠️ Available Tools

Models learn to select from 4 specialized legal research tools:

| Tool | Purpose | Parameters |
|------|---------|------------|
| `statute_retrieval` | Fetch SEC regulation text | `{"regulation": "str"}` |
| `case_law_search` | Search judicial precedents | `{"query": "str"}` |
| `compliance_checker` | Validate agreement clauses | `{"query": "str", "edgar_check": "str", "regulation": "str", "case_interpretation_check": "str"}` |
| `citation_validator` | Validate legal citations | `{"query": "str"}` |

### Turn kit on and start generating data


![alt text](imgs/data_kit.gif)

In [8]:
!synthetic-data-kit -c aws-config.yaml system-check

Loading config from: /Users/haasnani/Desktop/WWSO/RIV/AIM380/.venvNotebook/lib/python3.12/site-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /Users/haasnani/Desktop/WWSO/RIV/AIM380/.venvNotebook/lib/python3.12/site-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: aws-config.yaml
Config has LLM provider set to: api-endpoint
Environment variable check:
API_ENDPOINT_KEY: Not found
get_llm_provider returning: api-endpoint
API_ENDPOINT_KEY environment variable: Not found
API key source: Config file
⠸ Checking API endpoint access...INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
 API endpoint access confirmed
Using custom API base: http://localhost:8000/v1
Default model:  endpoint access...
Response from model: Hello there! It's great to have you here. If there's 
anything specific you're curious about or need assistance with, feel fre

In [8]:
# Ingest all cleaned data
!synthetic-data-kit -c aws-config.yaml ingest data/raw/edgar/
!synthetic-data-kit -c aws-config.yaml ingest data/raw/regulations/
!synthetic-data-kit -c aws-config.yaml ingest data/raw/cases/


Loading config from: /Users/haasnani/Desktop/WWSO/RIV/AIM380/.venvNotebook/lib/python3.12/site-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /Users/haasnani/Desktop/WWSO/RIV/AIM380/.venvNotebook/lib/python3.12/site-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: aws-config.yaml
Config has LLM provider set to: api-endpoint
Processing directory: data/raw/edgar/
Found 62 supported files to process
✓ agreement_000.txt
✓ agreement_001.txt
✓ agreement_002.txt
✓ agreement_003.txt
✓ agreement_004.txt
✓ agreement_005.txt
✓ agreement_006.txt
✓ agreement_007.txt
✓ agreement_008.txt
✓ agreement_009.txt
✓ agreement_010.txt
✓ agreement_011.txt
✓ agreement_012.txt
✓ agreement_013.txt
✓ agreement_014.txt
✓ agreement_015.txt
✓ agreement_016.txt
✓ agreement_017.txt
✓ agreement_018.txt
✓ agreement_019.txt
✓ agreement_020.txt
✓ agreement_021.txt
✓ agreement_022.txt
✓ agreement_023.txt
✓ ag

In [9]:
!synthetic-data-kit -c aws-config.yaml create data/parsed --type cot --verbose

Loading config from: /Users/haasnani/Desktop/WWSO/RIV/AIM380/.venvNotebook/lib/python3.12/site-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /Users/haasnani/Desktop/WWSO/RIV/AIM380/.venvNotebook/lib/python3.12/site-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: aws-config.yaml
Config has LLM provider set to: api-endpoint
get_llm_provider returning: api-endpoint
🔗 Using api-endpoint provider
Processing directory: data/parsed for cot generation
Found 101 cot files to process
Loading config from: aws-config.yaml━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% (0/101) 0:00:00
Config has LLM provider set to: api-endpoint━━━━━━━━━━━━━━━━   0% (0/101) 0:00:00
API_ENDPOINT_KEY from environment: Not found━━━━━━━━━━━━━━━━   0% (0/101) 0:00:00
Using API key: From config━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% (0/101) 0:00:00
Using API base URL: http://localhost:8000/v1━━━━━━━━━━━━━━━━   0% (0/10

In [10]:
from utils.data_processing import convert_to_jsonl
convert_to_jsonl(input_dir="data/generated/", output_file="data/final/sft/combined_training_data.jsonl")


Processing 101 files from directory: data/generated/
  ✓ agreement_000_cot_examples.json: 0 examples
  ✓ agreement_001_cot_examples.json: 2 examples
  ✓ agreement_002_cot_examples.json: 3 examples
  ✓ agreement_003_cot_examples.json: 3 examples
  ✓ agreement_004_cot_examples.json: 3 examples
  ✓ agreement_005_cot_examples.json: 3 examples
  ✓ agreement_006_cot_examples.json: 3 examples
  ✓ agreement_007_cot_examples.json: 3 examples
  ✓ agreement_008_cot_examples.json: 3 examples
  ✓ agreement_009_cot_examples.json: 3 examples
  ✓ agreement_010_cot_examples.json: 41 examples
  ✓ agreement_011_cot_examples.json: 3 examples
  ✓ agreement_012_cot_examples.json: 0 examples
  ✓ agreement_013_cot_examples.json: 3 examples
  ✓ agreement_014_cot_examples.json: 36 examples
  ✓ agreement_015_cot_examples.json: 3 examples
  ✓ agreement_016_cot_examples.json: 3 examples
  ✓ agreement_017_cot_examples.json: 3 examples
  ✓ agreement_018_cot_examples.json: 3 examples
  ✓ agreement_019_cot_examples.js

'data/final/sft/combined_training_data.jsonl'

In [2]:
# Add system prompt
prompt_system = """You are a securities law tool selection specialist.

Your task: 
1. Classify query type from 8 predefined categories
2. Select appropriate tools, tool input, and tool sequence. Provide reasoning for each tool choice.
3. Focus on connecting SEC regulations, EDGAR filings, and case law through expert tool selection decisions.
4. Output structured JSON format:
```json
"{\"Query analysis\": {\"Type\": \"[predefined_type]\", \"Information needed\": \"[specific_requirements]\", \"Tools\": [{\"Tool\": \"[tool_name]\", \"Parameters\": {[parameter_dict]}, \"Reasoning\": \"[why_this_tool]\"}, {\"Tool\": \"[tool_name_2]\", \"Parameters\": {[parameter_dict_2]}, \"Reasoning\": \"[why_this_tool_2]\"}]}}"
```

AVAILABLE TOOLS: statute_retrieval, case_law_search, compliance_checker, citation_validator

PREDEFINED TYPES: regulatory_definition, judicial_interpretation, compliance_validation, citation_verification, regulatory_compliance_analysis, judicial_compliance_assessment, cross_document_analysis, regulatory_interpretation_research"""



In [8]:
from utils.data_processing import add_system_prompt
add_system_prompt(prompt=prompt_system, input_file="data/final/sft/combined_training_data.jsonl")

Processed 100 examples...
Processed 200 examples...
Processed 300 examples...
Processed 400 examples...
Processed 500 examples...
Processed 600 examples...
Processed 700 examples...
Processed 800 examples...
Processed 900 examples...
Processed 1000 examples...
Processed 1100 examples...
✓ Added system prompt to 1170 examples
✓ Output written to: data/final/sft/combined_training_data_with_system.jsonl


In [1]:
from utils.data_processing import validate_and_report
validate_and_report(input_path="data/final/sft/combined_training_data_with_system.jsonl", fix=True)

💾 Validation report saved to: None
🔍 TRAINING DATA VALIDATION REPORT
📁 Files processed: 1
📊 Total examples: 1170
✅ Valid examples: 1107
❌ Errors: 63
⚠️ Warnings: 1

🚨 ERROR SUMMARY:
   JSON Parsing - Missing Comma: 53 occurrences
     • Line 92: Assistant response not valid JSON - Expecting ',' delimiter: line 1 column 564 (char 563)
     • Line 93: Assistant response not valid JSON - Expecting ',' delimiter: line 1 column 328 (char 327)
     • Line 94: Assistant response not valid JSON - Expecting ',' delimiter: line 1 column 271 (char 270)
     ... and 50 more

   Invalid Predefined Type: 5 occurrences
     • Line 132: Invalid Type 'judicial_interpretation_research', must be one of: {'regulatory_compliance_analysis', 'judicial_interpretation', 'cross_document_analysis', 'compliance_validation', 'regulatory_definition', 'regulatory_interpretation_research', 'citation_verification', 'judicial_compliance_assessment'}
     • Line 450: Invalid Type 'judicial_interpretation_research', must

In [2]:
from utils.data_processing import data_sft_rft

input_path="data/final/sft/combined_training_data_with_system.jsonl"
output_path="data/final/rft/combined_training_data_with_system.jsonl"

data_sft_rft(input_path=input_path, output_path=output_path)


Reading from: data/final/sft/combined_training_data_with_system.jsonl
Writing to: data/final/rft/combined_training_data_with_system.jsonl
Processed 100 records...
Processed 200 records...
Processed 300 records...
Processed 400 records...
Processed 500 records...
Processed 600 records...
Processed 700 records...
Processed 800 records...
Processed 900 records...
Processed 1000 records...
Processed 1100 records...
Transformation complete for combined_training_data_with_system.jsonl!
  Records processed: 1106
  Records failed: 0
  Output file: data/final/rft/combined_training_data_with_system.jsonl


### Train/Val/Test Split

The dataset is split into training (72%), validation (18%), and test (10%) sets to properly evaluate the model. 

In [3]:
# Split training (72%), validation (18%), and test (10%)
from utils.data_processing import split_jsonl

split_jsonl(input_file="data/final/sft/combined_training_data_with_system.jsonl", output_dir="data/final/sft", 
                train=0.72, val=0.10, test=0.18)

train: 796
val: 110
test: 200


In [4]:
# Split training (72%), validation (18%), and test (10%)
from utils.data_processing import split_jsonl

split_jsonl(input_file="data/final/rft/combined_training_data_with_system.jsonl", output_dir="data/final/rft", 
                train=0.72, val=0.10, test=0.18)

train: 796
val: 110
test: 200


### Nova offline evaluation dataset format SFT
The purpose of the evaluation process is to assess trained-model performance against benchmarks or custom dataset. The evaluation process typically involves steps to create evaluation recipe pointing to the trained model, specify evaluation datasets and metrics, submit a separate job for the evaluation, and evaluate against standard benchmarks or custom data. The evaluation process will output performance metrics stored in your Amazon S3 bucket.

RFT retains the same format as it's training set for offline evaluation

```json
{
   "system":"Content of the System prompt",
   "query":"Content of the user prompt",
   "response":"Content of the assistant answer"
}
```

In [1]:
from utils.data_processing import nova_offline_eval

nova_offline_eval(input_path="data/final/sft/test/test.jsonl")

Converting data/final/sft/test/test.jsonl -> data/final/sft/test/test_flat.jsonl
Conversion complete: 200 examples converted, 0 errors


### Upload curated datasets (SFT/RFT - train, test, val) to Amazon S3
This section sets up the necessary AWS credentials an S3 Client and a SageMaker session . You'll need proper IAM permissions to upload.

The code initializes a SageMaker session, sets up the IAM role, and configures the S3 bucket for storing training data and model artifacts.



In [ ]:
import sagemaker
import boto3
import os 

sess = sagemaker.Session()
sagemaker_session_bucket = None

if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
bucket_name = sess.default_bucket()
default_prefix = sess.default_bucket_prefix

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
import boto3

s3_client = boto3.client('s3')

# save train_dataset to s3 using our SageMaker session
if default_prefix:
    input_path = f"{default_prefix}/datasets/nova-sft-peft"
else:
    input_path = f"datasets/nova2_0-sft-peft"

# sft
train_dataset_s3_path_sft = f"s3://{bucket_name}/{input_path}/sft/train/dataset.jsonl"
val_dataset_s3_path_sft = f"s3://{bucket_name}/{input_path}/sft/val/dataset.jsonl"
test_dataset_s3_path_sft = f"s3://{bucket_name}/{input_path}/sft/test/gen_qa.jsonl"

s3_client.upload_file(
    "data/final/sft/train/train.jsonl", bucket_name, f"{input_path}/sft/train/dataset.jsonl"
)

s3_client.upload_file(
    "data/final/sft/val/val.jsonl", bucket_name, f"{input_path}/sft/val/dataset.jsonl"
)

s3_client.upload_file(
    "data/final/sft/test/test_flat.jsonl", bucket_name, f"{input_path}/sft/test/gen_qa.jsonl"
)

# rft
train_dataset_s3_path_rft = f"s3://{bucket_name}/{input_path}/rft/train/dataset.jsonl"
val_dataset_s3_path_rft = f"s3://{bucket_name}/{input_path}/rft/val/dataset.jsonl"
test_dataset_s3_path_rft = f"s3://{bucket_name}/{input_path}/rft/test/gen_qa.jsonl"

s3_client.upload_file(
    "data/final/rft/train/train.jsonl", bucket_name, f"{input_path}/rft/train/dataset.jsonl"
)

s3_client.upload_file(
    "data/final/rft/val/val.jsonl", bucket_name, f"{input_path}/rft/val/dataset.jsonl"
)

s3_client.upload_file(
    "data/final/rft/test/test.jsonl", bucket_name, f"{input_path}/rft/test/gen_qa.jsonl"
)

print(f"Training data uploaded to:")
print(train_dataset_s3_path_sft)
print(test_dataset_s3_path_sft)
print(val_dataset_s3_path_sft)
print(train_dataset_s3_path_rft)
print(test_dataset_s3_path_rft)
print(val_dataset_s3_path_rft)

In [ ]:
%store train_dataset_s3_path_sft
%store test_dataset_s3_path_sft
%store val_dataset_s3_path_sft
%store train_dataset_s3_path_rft
%store test_dataset_s3_path_rft
%store val_dataset_s3_path_rftf
